In [1]:
import numpy as np
import os 
path ="/kaggle/input/movie-review-polarity/txt_sentoken/"

In [2]:
import numpy as np
import os 
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
from os import listdir
from numpy import array

import urllib.request
import tarfile
# Define the URL of the dataset
url = "http://www.cs.cornell.edu/people/pabo/movie-review-data/review_polarity.tar.gz"

# Define the destination folder to save the dataset
destination_folder = "./review_polarity"

# Download the dataset
print("Downloading dataset...")
urllib.request.urlretrieve(url, "review_polarity.tar.gz")
print("Dataset downloaded successfully.")

# Extract the dataset
print("Extracting dataset...")
with tarfile.open("review_polarity.tar.gz", "r:gz") as tar:
    tar.extractall(destination_folder)
print("Dataset extracted successfully.")

Dataset downloaded successfully.
Extracting dataset...
Dataset extracted successfully.


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from nltk.corpus import stopwords 
from collections import Counter
import string
import re
from os import listdir
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.models import load_model

from tensorflow.keras.preprocessing.sequence import pad_sequences

In [6]:
import nltk
from nltk.corpus import stopwords

# Download stopwords and wordnet
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [7]:
stop_words = stopwords.words('english')
# i have to make a document and label s

def load_doc(filename):
    file = open(filename,'r')
    text = file.read()
    file.close()
    return text


    
def clean_doc(doc):
    
    table = str.maketrans(dict.fromkeys(string.punctuation))
    doc =  doc.translate(table)
    doc = doc.split()
    token = [word for word in doc   if word.isalpha()]
    token = [word for word in token if word not in stop_words] 
    token = [word for word in token if len(word)>1]
    
    return token 

In [8]:
min_occurence =2
def process_doc(directory,vocab):
    for filename in os.listdir(directory):
        
        doc =  load_doc(os.path.join(directory,filename))
        token = clean_doc(doc)
        vocab.update(token)

In [9]:
path = './review_polarity/txt_sentoken/'

vocab = Counter()

pos_tokens = process_doc(path+'pos',vocab)
neg_tokens = process_doc(path+'neg',vocab)

vocab.most_common(10)
tokens = [k for k,v in vocab.items() if v>=min_occurence]

def save_list(lines,filename):
    data = '\n'.join(lines)
    file = open(filename,'w')
    file.write(data)
    file.close()
save_list(tokens,'vocab.txt')

vocab = load_doc('vocab.txt')
vocab = set(vocab.split())

# 新段落

In [10]:
def updated_clean_doc(doc,vocab):
    
    table = str.maketrans(dict.fromkeys(string.punctuation))
    doc =  doc.translate(table)
    doc = doc.split()
    token = [word for word in doc   if word.isalpha()]
    token = [word for word in token if word not in stop_words] 
    token = [word for word in token if len(word)>1]
    token = [word for word in token if word in vocab]
    return token 
def updated_process_doc(directory,vocab):
    documents = list()
    for filename in os.listdir(directory):
        
        doc =  load_doc(os.path.join(directory,filename))
        token = updated_clean_doc(doc,vocab)
        lines = ' '.join(token)
        documents.append(lines)
    return documents

In [15]:
path = './review_polarity/txt_sentoken/'
pos =  updated_process_doc(path+'pos',vocab)
neg =  updated_process_doc(path+'neg',vocab)
document = pos+ neg 
labels = [1]* len(pos ) + [0]*len(neg)

assert len(labels) == len(pos)+len(pos)
def create_tokenizer(doc):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(doc)
    return tokenizer
def create_encoding(doc,tokenizer,length):
    tokens = tokenizer.texts_to_sequences(doc)
    padded = pad_sequences(tokens,length,padding='post')
    return padded
max_length = max([len(i.split()) for i in document])
max_length

tokenizer = create_tokenizer(document)
padded = create_encoding(document,tokenizer,max_length)

padded = np.array(padded)
labels = np.array(labels)
vocab_size = len(tokenizer.word_index) +1

In [16]:
def define_model(vocab_size,max_length):

    model = Sequential() 
    model.add(Embedding(vocab_size,100,input_length=max_length))

    model.add(Conv1D(32,kernel_size=8,activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    model.add(Dense(10,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    model.summary() 
    plot_model(model,to_file='model.png',show_shapes=True)
    return model
os.environ["KMP_SETTINGS"] = "false"
model = define_model(vocab_size,max_length)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(padded,labels,epochs=10,validation_split=0.3)

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 1319, 100)         2714000   
                                                                 
 conv1d_2 (Conv1D)           (None, 1312, 32)          25632     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 656, 32)          0         
 1D)                                                             
                                                                 
 flatten_2 (Flatten)         (None, 20992)             0         
                                                                 
 dense_4 (Dense)             (None, 10)                209930    
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                      

In [17]:
model.save('model.h5')
def predict_sentiment(review,vocab,tokenizer,max_length,model):
    
    line = updated_clean_doc(review,vocab)
    padded = create_encoding([line],tokenizer,max_length)
    
    yhat = model.predict(padded)
    percent_pos = yhat[0,0]
    if round(percent_pos) == 0:
        return (1-percent_pos), 'NEGATIVE' 
    return percent_pos, 'POSITIVE'
test_file = './review_polarity/txt_sentoken/neg/cv001_19502.txt'

test_doc = load_doc(test_file)
loaded_model = load_model('model.h5')
predict_sentiment(test_doc,vocab,tokenizer,max_length,loaded_model)

1/1 [==============================] - 0s 134ms/step


(0.9996484140865505, 'NEGATIVE')

In [18]:
# Set the maximum vocabulary size to 10,000
max_vocab_size = 10000

# Create a new tokenizer with the reduced vocabulary size
tokenizer_trunc = Tokenizer(num_words=max_vocab_size)
tokenizer_trunc.fit_on_texts(document)

# Encode the sequences using the reduced vocabulary
padded_trunc_vocab = create_encoding(document, tokenizer_trunc, max_length)


In [19]:
def define_model(vocab_size,max_length):

    model = Sequential() 
    model.add(Embedding(vocab_size,100,input_length=max_length))

    model.add(Conv1D(32,kernel_size=8,activation='relu'))
    model.add(MaxPooling1D(pool_size=2))

    model.add(Flatten())

    model.add(Dense(10,activation='relu'))

    model.add(Dense(1,activation='sigmoid'))

    model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

    model.summary() 
    plot_model(model,to_file='model.png',show_shapes=True)
    return model
os.environ["KMP_SETTINGS"] = "false"
model = define_model(vocab_size,max_length)

model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

model.fit(padded,labels,epochs=10,validation_split=0.3)

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 1319, 100)         2714000   
                                                                 
 conv1d_3 (Conv1D)           (None, 1312, 32)          25632     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 656, 32)          0         
 1D)                                                             
                                                                 
 flatten_3 (Flatten)         (None, 20992)             0         
                                                                 
 dense_6 (Dense)             (None, 10)                209930    
                                                                 
 dense_7 (Dense)             (None, 1)                 11        
                                                      

In [ ]:
# Set the maximum vocabulary size to 10,000
max_vocab_size = 10000

# Create a new tokenizer with the reduced vocabulary size
tokenizer_trunc = Tokenizer(num_words=max_vocab_size)
tokenizer_trunc.fit_on_texts(document)

# Encode the sequences using the reduced vocabulary
padded_trunc_vocab = create_encoding(document, tokenizer_trunc, max_length)

# Train a model using the reduced vocabulary
model_trunc_vocab = ...
model_trunc_vocab.fit(padded_trunc_vocab, labels, ...)


In [22]:
import numpy as np
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Load the pre-trained GloVe embeddings
glove_dir = '/Users/watson/Desktop/algonquin/nlp/glove.6B/'
embedding_dim = 100
embeddings_index = {}
with open(os.path.join(glove_dir, 'glove.6B.100d.txt'), encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

# Load and preprocess the IMDB dataset
max_features = 10000
maxlen = 200
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train = pad_sequences(x_train, maxlen=maxlen)
x_test = pad_sequences(x_test, maxlen=maxlen)

# Prepare the embedding matrix
embedding_matrix = np.zeros((max_features, embedding_dim))
for word, i in word_index.items():
    if i >= max_features:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# Define the model architecture
model = Sequential()
model.add(Embedding(max_features, embedding_dim, input_length=maxlen, weights=[embedding_matrix], trainable=False))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

# Compile and train the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))
